# Introducción a la recuperación de información
Autor: Eric S. Tellez <eric.tellez@infotec.mx><br/>

La recuperación de información se crea a partir de la necesidad de simplificar el acceso y revisión de documentos en grandes colecciones. Estas colecciones pueden ser homogeneas o heterogeneas, tanto en su contenido como en su formato. En su inicio, se consideraban colecciones unicamente de texto pero las necesidades de información se han diversificado, y ahora es común encontrar sistemas de recuperación de información sobre otros datos como imagenes o videos, o include multimodales, esto es que puedan usar diferentes tipos de objetos. Un objeto puede ser un documento de texto, una imagen, o cualquier otro tipo de datos que se desee tener acceso. 

Mantener un sistema de información homogeneo puede simplificar su mantenimiento enormemente, por lo que si es posible, se puede intentar mantener cierta homegeneidad. Para sistemas de fuentes abiertas como puede ser la web, esto será posible.

En general, se puede ver un sistema de recuperación de información en tres grandes partes:

1. Recolección de datos, normalización y modelado matemático.
2. Indexamiento, interpretación de consultas y optimización
3. Agregación y filtrado de resultados, presentación de los mismos


En este curso se visitarán parcialmente todas estas partes. En general se usaran conjuntos de datos previamente recolecados, aunque se invita a usar sus propias colecciones de tal forma que puedan sacar provecho de los temas de manera más amplia. La normalización puede ir desde el simple preprocesamiento de los datos hasta manipulaciones y transformaciones dependientes del dominio. Una vez aplicado el modelado adecuado a las colecciones, las representaciones matemáticas suelen ser vectores de alta dimensión para cada objeto.

En cuanto al indexamiento se utilizarán dos tipos de algoritmos, búsqueda mediante índices invertidos y búsqueda por índices métricos. Ambas tienen sus nichos de aplicación y usarlas adecuadamente requiere conocer los problemas y sus modelados.

La presentación de los resultados puede ser tan simple como una lista de resultados más relevantes y una pequeña muestra del objeto, o más complejo que requiera alguna técnica de visualización. Todo esto dependerá del dominio de aplicación y la naturaleza del sistema de recuperación de información.


# Ejemplos de sistemas de recuperación información

- Google,scholar google, google images, google news, youtube...
- Bing!
- Yahoo search
- Yandex
- DuckDuckGo
- ...

También es común que el sistema de recuperación de información no sea el principal producto, si no más bien un complemento imperdible
como es el caso de los sistemas de streaming de videos, películas, música, redes sociales, etc.

# Ejemplos de implementación de search engines open source
- [Apache Lucene](https://lucene.apache.org/)
- [Apache SOLR](https://solr.apache.org/)
- [ElasticSearch](https://www.elastic.co/es/what-is/elasticsearch)
- [hnswlib](https://github.com/nmslib/hnswlib)
- [FAISS](https://github.com/facebookresearch/faiss)
- [TextSearch.jl](https://github.com/sadit/TextSearch.jl)
- [InvertedFiles.jl](https://github.com/sadit/InvertedFiles.jl)
- [SimilaritySearch.jl](https://github.com/sadit/SimilaritySearch.jl)


# Ejemplo


Construcción de un índice de búsqueda para una colección de mensajes de Twitter. Los mensajes se pueden ver como documentos cortos. Se preprocesan y se modalen en vectores mediante un vocabulario. Después se construye el índice con dichos vectores y se hace posible la búsqueda eficiente. Note que existen etiquetas que se pueden usar para otras tareas; de manera más genérica, se pueden ver como metadatos asociados a los elementos del corpus.

Nota: La primera vez que se corra este notebook los paquetes necesarios se instalaran de manera automática, es posible que tome un tiempo considerable.



In [1]:
using Pkg
Pkg.activate(".")

using TextSearch, InvertedFiles, SimilaritySearch, LinearAlgebra, HypertextLiteral

  Activating project at `~/Cursos/IR-2024/Unidades`


## Funciones para leer y modelar el corpus

In [2]:
function text_model_and_vectors(corpus)
    textconfig = TextConfig(group_usr=true, group_url=true, del_diac=true, lc=true, group_num=true, nlist=[1], qlist=[])
    voc = Vocabulary(textconfig, corpus)
    model = VectorModel(IdfWeighting(), TfWeighting(), voc)    
    vectors = vectorize_corpus(model, corpus)

    (; textconfig, model, vectors)
end


text_model_and_vectors (generic function with 1 method)

In [3]:
display(@htl "<h1>Cargando el corpus</h1>")

include("read_datasets.jl")
D, _ = read_emojispace()
T = text_model_and_vectors(D.corpus)

@show unique(D.labels), T.model

<h1>Cargando el corpus</h1>

(unique(D.labels), T.model) = (["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡", "🤓", "🙂", "❤", "😠", "😳", "😭", "👌", "🌚", "😅", "😐", "😋", "😕", "😉", "😢", "😱", "💕", "🤣", "😪", "😎", "😏", "😻", "🤭", "💜", "🙈", "🙏", "😥", "😀", "😤", "😩", "😘", "😊", "🙄", "🤗", "😜", "😑", "✨", "😰", "😂", "🎶", "🙃", "🙊", "🤤", "👏", "😁", "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})


(["♡", "😒", "💖", "😴", "😍", "😓", "😬", "💙", "😃", "😡"  …  "😞", "😣", "😌", "🤔", "♥", "💔", "👀", "😔", "😈", "😫"], {VectorModel
    global_weighting: IdfWeighting()
    local_weighting: TfWeighting()
    vocsize: 41704
    trainsize=45000
    maxoccs=84742                                    
})

### Note que las etiquetas son emojis, que hay 45k ejemplos y un vocabulario de más de 42k tokens. A continuación se construirá el índice invertido usando el paquete <https://github.com/sadit/TextSearch.jl>, claramente, el código solo funcionará con dicha implementación, pero el procedimiento general aplicaría a otras implementaciones.

In [4]:
invfile = WeightedInvertedFile(length(T.model.voc))
append_items!(invfile, VectorDatabase(T.vectors))

{WeightedInvertedFile{Nothing, SimilaritySearch.AdjacencyLists.AdjacencyList{IdWeight}} vocsize=41704, n=45000}

In [5]:
function search_and_display(invfile, q, k, D, T)
    res = KnnResult(k)
    @time search(invfile, vectorize(T.model, q), res)
    L = []
    for (i, p) in enumerate(res)
        dist = round(p.weight, digits=2)
        push!(L, @htl """<tr><td>$i</td> <td>$p.id => $dist</td> <td>$(D.labels[p.id])</td> <td>$(D.corpus[p.id])</td> </tr>""")
    end

    display(@htl """
    <h3>Resultados para la consulta "$q"</h3>
    <table>
    $L
    </table>
    """)
end

search_and_display (generic function with 1 method)

In [6]:
search_and_display(invfile, "no importa distancia", 5, D, T)
search_and_display(invfile, "cuando nos vemos?", 5, D, T)
search_and_display(invfile, "feliz cumpleaños?", 3, D, T)
search_and_display(invfile, "ola k ase", 3, D, T)
search_and_display(invfile, "mi humilde opinión", 10, D, T)

  0.032347 seconds (1.21 k allocations: 56.797 KiB, 99.05% compilation time)


1,"IdWeight(0x00008a0f, 0.033524435f0).id => 0.03",💕,No importa la distancia _emo _usr _url
2,"IdWeight(0x000060ef, 0.38421434f0).id => 0.38",😓,Me caga la distancia _emo
3,"IdWeight(0x0000912b, 0.5246081f0).id => 0.52",😃,No importa cuando... _emo _emo _url
4,"IdWeight(0x00000508, 0.5725885f0).id => 0.57",😥,"tiempo y distancia, quizás sea eso... _emo"
5,"IdWeight(0x00008e49, 0.5733391f0).id => 0.57",😑,_usr A ti no te importa _emo


  0.000307 seconds (25 allocations: 3.602 KiB)


1,"IdWeight(0x00007566, 0.22503656f0).id => 0.23",😂,Nos vemos!! _emo _emo _emo _emo _url
2,"IdWeight(0x000054f5, 0.30652022f0).id => 0.31",😜,_usr En la tarde nos vemos _emo _emo _emo
3,"IdWeight(0x00002374, 0.33539587f0).id => 0.34",😻,Nos vemos pronto _usr _usr _emo _emo _url
4,"IdWeight(0x00005e69, 0.4330908f0).id => 0.43",😍,_usr Pronto nos vemos hermana _emo _emo
5,"IdWeight(0x00009126, 0.45266888f0).id => 0.45",😕,Apenas van _num días que no nos vemos _emo


  0.000158 seconds (23 allocations: 3.547 KiB)


1,"IdWeight(0x000067bc, 0.105059296f0).id => 0.11",❤,_usr Feliz Cumpleaños _emo
2,"IdWeight(0x00009d25, 0.10506016f0).id => 0.11",😁,Feliz Cumpleaños _emo _emo _usr
3,"IdWeight(0x00002244, 0.105065644f0).id => 0.11",🙊,Feliz Cumpleaños _usr _emo _emo _emo _emo _emo


  0.000039 seconds (22 allocations: 3.406 KiB)


1,"IdWeight(0x0000aa8f, 0.6446407f0).id => 0.64",😎,"_usr _usr Siempre, todo y todos los días de la semana desde ase muchoooooossss años _emo _emo"
2,"IdWeight(0x00005c21, 0.7058066f0).id => 0.71",😤,_usr k bueno k tu mizmo lo reconozcAszzzz _emo
3,"IdWeight(0x00009619, 0.7076125f0).id => 0.71",😃,"Despertares!! (By OLA) # _emo _emo en Distrito Federal, Mexico _url"


  0.000182 seconds (23 allocations: 3.523 KiB)


1,"IdWeight(0x00000864, 0.32944086f0).id => 0.33",🙏,"Bueno, cada quien su vida, ya se, solo es una humilde opinión !! _emo _emo _emo"
2,"IdWeight(0x00008bea, 0.5080328f0).id => 0.51",😍,_usr es la persona más humilde que conozco _emo
3,"IdWeight(0x0000805a, 0.6217729f0).id => 0.62",🤔,_usr Bajo tu opinión fue o no falta? _emo
4,"IdWeight(0x00006ee9, 0.6222807f0).id => 0.62",😊,Gracias por compartir la información y tu opinión _emo _url
5,"IdWeight(0x00007bb3, 0.6478042f0).id => 0.65",😉,Cuantos Likes para mi humilde oficina _usr _emo _emo @ CocoBongo _url
6,"IdWeight(0x0000159c, 0.663409f0).id => 0.66",🤗,"Para ser feliz, no escuches la opinión de los demás _emo"
7,"IdWeight(0x000031a9, 0.6741618f0).id => 0.67",👏,_usr Totalmente de acuerdo contigo en todo. Comparto tu opinión _emo _emo _emo _emo
8,"IdWeight(0x000003de, 0.72290635f0).id => 0.72",😌,"Tu opinión no me interesa más. No tengo nada que demostrarte, y si piensas lo peor de mi, MEJOR! _emo"
9,"IdWeight(0x00009ab1, 0.72665346f0).id => 0.73",😑,"Lo siento pero mi tez humilde no me deja ser guapo y estar bueno, eso sin mencionar la rodilla que me chingue! _emo _emo"
10,"IdWeight(0x00002d45, 0.7458333f0).id => 0.75",😃,Interesante opinión y espero que sigas escribiendo _usr en mejores espacios !suerte!! _emo _emo _emo _url


### También es posible consultar elementos del mismo corpus, lo cual permité inspeccionar la estructura interna de la colección. Esto será de utilidad más adelante en el curso.

In [7]:
for i in 1:10
    qID = rand(1:length(D.corpus))
    search_and_display(invfile, D.corpus[qID], 7, D, T)
end

  0.006924 seconds (59 allocations: 5.500 KiB)


1,"IdWeight(0x00001d91, 2.3174108f-7).id => 0.0",🤗,"Tener el poder de destruir a tus enemigos y no hacerlo, se siente verga _emo"
2,"IdWeight(0x00005770, 0.70924336f0).id => 0.71",😊,Que bien se siente tener la razón _emo _emo _emo
3,"IdWeight(0x00007735, 0.7130483f0).id => 0.71",😈,_usr Cuando Úrsula llena de poder e irá se hace gigante y quiere destruir todo y a todos. _emo _url
4,"IdWeight(0x0000a41a, 0.7400262f0).id => 0.74",😩,Primera vez que llevo mi maqueta así y siento que se me va a destruir _emo _emo
5,"IdWeight(0x00003f80, 0.7518793f0).id => 0.75",🙈,Que bonito se siente dormir tus _num hrs _emo
6,"IdWeight(0x000000e2, 0.75284344f0).id => 0.75",😞,Que mal se siente. _emo _emo
7,"IdWeight(0x00006194, 0.757171f0).id => 0.76",💜,Y se siente bien bonito _emo _emo _emo _url


  0.008671 seconds (57 allocations: 5.453 KiB)


1,"IdWeight(0x000094fd, -1.6425588f-8).id => -0.0",😀,Esperando ya para comprar la edición de colección. _emo _emo _emo _emo _url
2,"IdWeight(0x0000a8bc, 0.4016952f0).id => 0.4",♥,Para la colección _emo _emo _url
3,"IdWeight(0x00005cc3, 0.5273127f0).id => 0.53",🙈,Dos más a la colección _emo _emo _url
4,"IdWeight(0x00007672, 0.6449609f0).id => 0.64",😌,_usr Ya te está esperando _emo _emo
5,"IdWeight(0x0000439d, 0.7048287f0).id => 0.7",😀,_usr La azul Edición El principito _emo _url
6,"IdWeight(0x00009810, 0.70753354f0).id => 0.71",🙂,Ando esperando _emo
7,"IdWeight(0x00008b9c, 0.715319f0).id => 0.72",🤔,Mañana que me voy a comprar _emo


  0.007395 seconds (67 allocations: 5.891 KiB)


...

1,"IdWeight(0x00001362, 6.749433f-8).id => 0.0",😞,_usr Tú quieres que me de el soponcio _emo _url
2,"IdWeight(0x00006859, 0.45605382f0).id => 0.46",🙄,_usr tu que quieres _emo
3,"IdWeight(0x0000ae6c, 0.5068261f0).id => 0.51",😡,"Como diría mi abuelo, me está dando el soponcio. _emo"
4,"IdWeight(0x00000a45, 0.5726747f0).id => 0.57",😏,_usr por que no quieres _emo
5,"IdWeight(0x0000a07d, 0.6048379f0).id => 0.6",🙊,_usr Porque quieres _emo _emo
6,"IdWeight(0x00008410, 0.6140172f0).id => 0.61",😏,Los quieres? _emo _url
7,"IdWeight(0x000051db, 0.6187732f0).id => 0.62",😢,"Me dices, que tú ya no me quieres _emo"


  0.002746 seconds (48 allocations: 4.852 KiB)


1,"IdWeight(0x00008cd4, 2.3260327f-8).id => 0.0",😏,_usr Te gusta lo difíciiiiiiiil _emo _emo _emo _emo _emo _emo
2,"IdWeight(0x00007fad, 0.6201663f0).id => 0.62",🙏,_usr _usr me gusta _emo _emo _emo _emo _emo _emo
3,"IdWeight(0x000090c7, 0.65856946f0).id => 0.66",🤭,_usr _emo si me gusta
4,"IdWeight(0x0000a59d, 0.67546445f0).id => 0.68",😅,A ver si te gusta _emo _emo _url
5,"IdWeight(0x000050d8, 0.6840733f0).id => 0.68",😋,_usr _usr _emo el que me gusta....
6,"IdWeight(0x00000a77, 0.68818736f0).id => 0.69",🤭,_usr Así me gusta _emo
7,"IdWeight(0x000042dc, 0.69254804f0).id => 0.69",🤗,Esta me gusta _emo _url


  0.002958 seconds (44 allocations: 4.617 KiB)


1,"IdWeight(0x00003d31, -5.8978458f-9).id => -0.0",😋,Que chulos están mis primos _emo _emo _emo _emo _url
2,"IdWeight(0x0000472b, 0.46542224f0).id => 0.47",😜,Primos... _emo _url
3,"IdWeight(0x0000028f, 0.5938023f0).id => 0.59",😳,_usr _usr _emo ¿son primos? _emo
4,"IdWeight(0x00008308, 0.6362801f0).id => 0.64",😃,¡Cena de primos! _emo
5,"IdWeight(0x000028ed, 0.6378803f0).id => 0.64",🤔,Mis primos usan mas mi carro que yo _emo
6,"IdWeight(0x0000a264, 0.6566216f0).id => 0.66",🙊,"_usr En el estadio!! Así infecte a todo el mundo, yo no dejo de apoyar a mis chulos _emo _emo"
7,"IdWeight(0x00000762, 0.66471934f0).id => 0.66",😻,"Ojitos chiquitos, bonitos, chulos de preciosos _emo"


# Actividades

- Revisión de los servicios y empresas mencionados 
- Revisión de los proyectos open source mencionados
- Cargue este notebook y modifique las consultas (necesita Jupyter y el kernel de Julia <https://github.com/JuliaLang/IJulia.jl>)
- Revise la bibliografía sobre recuperación de información, en especial [^SMR2008] se encuentra libre desde <https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf>

# Bibliografía
- [SMR2008] Schütze, H., Manning, C. D., & Raghavan, P. (2008). Introduction to information retrieval (Vol. 39, pp. 234-65). Cambridge: Cambridge University Press.
- [BYN1999] Baeza-Yates, R., & Ribeiro-Neto, B. (1999). Modern information retrieval (Vol. 463). New York: ACM press.